In [1]:
import pickle
import json
import oyaml as yaml
from collections import OrderedDict
from pathlib import Path
import sys

p = Path('.')  / 'cloudmesh' / 'bookmanagerservice' / 'service'
sys.path.append(str(p.absolute()))

import getdata

In [8]:
bks = getdata.getBooks(onlybooks=True)
tstbk = 'Cloud Computing'
bkinfo = getdata.getBooks(onlybooks = False, filename = bks[tstbk])

#All Data
mainData = bkinfo[tstbk]['data']
links = bkinfo[tstbk]['links']
metadata = bkinfo[tstbk]['metadata']
data = pickle.load( open( "returnedData.pkl", "rb" ))

![alt text](sample.jpg "Title")

In [9]:
""" https://stackoverflow.com/a/8661021 """
#def represent_dictionary_order(self, dict_data):
#    return self.represent_mapping('tag:yaml.org,2002:map', dict_data.items())
#def setup_yaml():
#    yaml.add_representer(OrderedDict, represent_dictionary_order)
#setup_yaml()    

' https://stackoverflow.com/a/8661021 '

In [10]:
def rec(data, vals):
    lst2 = []
    for v in vals:
        lst = []
        for d in data:
            if d['parent'] == v:
                lst.append(d['id'])
        if len(lst) > 0:
            g = rec(data,lst)
            lst2.append(({v : g}))
        else:
            lst2.append(v)
    return lst2        

In [11]:
final = {}
for d in data:
    if d['parent'] == '#':
        final[d['id']] = d['children']

In [12]:
temp =  {}
temp2 = {}

for keys, values in final.items():
    lst = rec(data,values)
    temp[keys] = lst
    
    
#Missing PART
#convert names in temp to links
def unpack(content):
    nList = []
    for k, v in content.items():
        for value in v:
            if isinstance(value, dict):
                nList += (unpack(value))
            else:
                nList.append(value)
    return nList
    
idList = unpack(temp)
temp = str(temp)
for item in idList:
    for i in range(len(mainData)):
        t = mainData[i]
        if item == t['id']:
            temp = temp.replace(item, links[i])

temp = eval(temp)

#
temp2['Metadata'] = metadata
temp2['BOOK'] = temp
temp2 = temp2

In [13]:
fdata = yaml.load(json.dumps(temp2),Loader = yaml.SafeLoader)

In [14]:
print(yaml.dump(fdata))

bookTitle = metadata['title']
with open(bookTitle + '.yaml', 'w+') as f:
    yaml.dump(fdata, f)

Metadata:
  image: cover.png
  title: Cloud Computing
  subtitle: ''
  author: Gregor von Laszewski
  subauthor: Editor
  email: laszewski@gmail.com
  url: https://cloudmesh-community.github.io/book/vonlaszewski-cloud.epub
  description: Proposed Lecture content (subject to change)
  abstract: Proposed Lecture content (subject to change)
  keywords: Cloud, virtual machines, containers, cloud engenering
  stylesheet: epub.css
  dest: ./dest/book
  filename: vonLaszewski-cloud.epub
  rights: (c) Gregor von Laszewski, 2018, 2019
BOOK:
  REST:
  - https://raw.githubusercontent.com/cloudmesh-community/book/master/chapters/rest/rest.md
  - REST/OPENAPI_3.0:
    - https://raw.githubusercontent.com/cloudmesh-community/book/master/chapters/rest/rest-spec.md
    - https://raw.githubusercontent.com/cloudmesh-community/book/master/chapters/rest/swagger-introspection.md
    - https://raw.githubusercontent.com/cloudmesh-community/book/master/chapters/rest/rest-ai.md
    - REST/OPENAPI_2.0:
      - h